# Analyse 3D des Chansons de Rap : Exploration de la Complexité Lexicale, du Message Politique et de la Vulgarité

L'analyse présentée dans ce notebook explore les données de chansons de rap issues du top 50 français, en se concentrant sur trois aspects clés : la complexité lexicale, le message politique et la vulgarité. Les données proviennent du dataset disponible à l'URL : https://huggingface.co/spaces/alihmaou/parolesparolesparoles/resolve/main/genius_top_50_fr_rap_pop_rock_analyzed.parquet. Ce dataset enrichi fournit des informations détaillées sur les chansons, notamment leur titre, artiste, année de sortie, ainsi que des analyses sur leur complexité lexicale, thèmes principaux et tonalité émotionnelle.

L'objectif de cette analyse est de créer un diagramme en 3D qui illustre l'évolution des chansons de rap en fonction de l'année, de l'artiste, de la complexité lexicale, du message politique et de la vulgarité. Cette visualisation permettra d'identifier les tendances et les corrélations entre ces différents aspects, offrant ainsi une compréhension plus approfondie de l'évolution du genre musical.

## Méthodologie

La méthodologie utilisée dans cette analyse implique plusieurs étapes clés. Tout d'abord, les données sont filtrées pour ne conserver que les chansons de rap. Ensuite, les variables de complexité lexicale, de message politique et de vulgarité sont transformées pour permettre leur utilisation dans l'analyse. Les données sont ensuite agrégées par artiste et par année, afin de calculer les moyennes de ces variables. Finalement, un diagramme en 3D animé est créé en utilisant la bibliothèque Plotly, permettant de visualiser l'évolution des chansons de rap dans le temps.

Les résultats de cette analyse sont présentés sous forme de visualisation interactive, facilitant l'exploration et la compréhension des données. Cette approche permet de mettre en évidence les tendances et les corrélations entre les différentes variables, offrant ainsi une vision plus claire de l'évolution du rap français.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [1]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://huggingface.co/spaces/alihmaou/parolesparolesparoles/resolve/main/genius_top_50_fr_rap_pop_rock_analyzed.parquet", safe_mode=True)
print("✅ Données chargées avec succès")

## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [2]:
# Exécution de la requête
df = run_query(""" WITH 
  -- Préparation des données pour les chansons de rap
  rap_data AS (
    SELECT 
      "artiste",
      CAST("annee" AS VARCHAR) AS "annee",
      CASE "complexite_lexicale"
        WHEN 'faible' THEN 1
        WHEN 'moyen' THEN 2
        WHEN 'élevé' THEN 3
        ELSE 0
      END AS complexite_lexicale_value,
      CASE "message_politique"
        WHEN 'faible' THEN 1
        WHEN 'moyen' THEN 2
        WHEN 'élevé' THEN 3
        ELSE 0
      END AS message_politique_value,
      "vulgarite"
    FROM 
      loaded_dataset
    WHERE 
      "style_musique" = 'rap'
  ),
  
  -- Agrégation par artiste et par annee
  aggregated_data AS (
    SELECT 
      "annee",
      "artiste",
      AVG(complexite_lexicale_value) AS avg_complexite_lexicale,
      AVG(message_politique_value) AS avg_message_politique,
      AVG("vulgarite") AS avg_vulgarite
    FROM 
      rap_data
    GROUP BY 
      ALL
  )

-- Sélection des données pour le diagramme
SELECT 
  "annee",
  "artiste",
  avg_complexite_lexicale,
  avg_message_politique,
  avg_vulgarite
FROM 
  aggregated_data
ORDER BY 
  "annee", "artiste" """)
print(f"Résultats : {len(df)} lignes")
df.head()

## 📈 Visualisation

La bibliothèque principale utilisée est Plotly, qui permet de créer des visualisations interactives et animées. Plotly est adaptée ici pour représenter les données en 3D avec une animation temporelle, facilitant la compréhension des évolutions des caractéristiques des chansons de rap au fil des ans. Cela permet une exploration intuitive et détaillée des données.

In [3]:
import pandas as pd
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Agrégation par année et artiste
df_agg = df.groupby(['artiste', 'annee']).agg({
    'avg_complexite_lexicale': 'mean',
    'avg_message_politique': 'mean',
    'avg_vulgarite': 'mean'
}).reset_index()

# Création du graphique 3D animé avec slider d'année
dataviz = px.scatter_3d(
    df_agg,
    x='avg_complexite_lexicale',
    y='avg_message_politique',
    z='avg_vulgarite',
    color='artiste',
    hover_name='artiste',
    animation_frame='annee',
    size_max=10,
    color_discrete_sequence=px.colors.qualitative.Set3,
    hover_data={
        'annee': True,
        'avg_complexite_lexicale': ':.2f',
        'avg_message_politique': ':.2f',
        'avg_vulgarite': ':.2f'
    }
)

# Configuration du layout pour une meilleure présentation
dataviz.update_layout(
    title={
        'text': "Analyse 3D des chansons de rap par artiste et année",
        'font': {'size': 20, 'family': 'Arial'}
    },
    scene=dict(
        xaxis_title='Complexité lexicale',
        yaxis_title='Message politique',
        zaxis_title='Vulgarité',
        aspectmode='cube'
    ),
    margin=dict(l=20, r=20, t=80, b=20),
    legend_title_text='Artiste',
    plot_bgcolor='rgba(0,0,0,0)'
)

# Ajout des transitions douces entre les frames
dataviz.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 750
dataviz.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1500
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*